In [1]:
from typing import TypeVar, Generic, Iterator, Callable, Optional, List, Any, Union
from functools import reduce
import math

In [2]:
T = TypeVar('T')
U = TypeVar('U')
V = TypeVar('V')

In [3]:
class LazyCollection(Generic[T]):
    """
    A custom collection class that provides lazy evaluation of expensive transformations.
    Operations are composable and memory usage scales with output size, not input size.
    """
    
    def __init__(self, source: Union[Iterator[T], List[T], 'LazyCollection[T]']):
        """
        Initialize with an iterator, list, or another LazyCollection.
        """
        if isinstance(source, LazyCollection):
            self._iterator = source._iterator
        elif isinstance(source, list):
            self._iterator = iter(source)
        else:
            self._iterator = source
    
    def __iter__(self) -> Iterator[T]:
        """Make the collection iterable."""
        return self._iterator
    
    def map(self, func: Callable[[T], U]) -> 'LazyCollection[U]':
        """
        Apply a mapping function to each element lazily.
        
        Args:
            func: Function to apply to each element
            
        Returns:
            New LazyCollection with mapped elements
        """
        def map_iterator():
            for item in self._iterator:
                yield func(item)
        
        return LazyCollection(map_iterator())
    
    def filter(self, predicate: Callable[[T], bool]) -> 'LazyCollection[T]':
        """
        Filter elements based on a predicate lazily.
        
        Args:
            predicate: Function that returns True for elements to keep
            
        Returns:
            New LazyCollection with filtered elements
        """
        def filter_iterator():
            for item in self._iterator:
                if predicate(item):
                    yield item
        
        return LazyCollection(filter_iterator())
    
    def flat_map(self, func: Callable[[T], Iterator[U]]) -> 'LazyCollection[U]':
        """
        Apply a function that returns iterators and flatten the results.
        
        Args:
            func: Function that returns an iterator for each element
            
        Returns:
            New LazyCollection with flattened results
        """
        def flat_map_iterator():
            for item in self._iterator:
                yield from func(item)
        
        return LazyCollection(flat_map_iterator())
    
    def take(self, n: int) -> 'LazyCollection[T]':
        """
        Take only the first n elements.
        
        Args:
            n: Number of elements to take
            
        Returns:
            New LazyCollection with first n elements
        """
        def take_iterator():
            count = 0
            for item in self._iterator:
                if count < n:
                    yield item
                    count += 1
                else:
                    break
        
        return LazyCollection(take_iterator())
    
    def skip(self, n: int) -> 'LazyCollection[T]':
        """
        Skip the first n elements.
        
        Args:
            n: Number of elements to skip
            
        Returns:
            New LazyCollection without first n elements
        """
        def skip_iterator():
            count = 0
            for item in self._iterator:
                if count >= n:
                    yield item
                count += 1
        
        return LazyCollection(skip_iterator())
    
    def reduce(self, func: Callable[[U, T], U], initial: Optional[U] = None) -> U:
        """
        Reduce the collection to a single value.
        
        Args:
            func: Reduction function
            initial: Initial value (optional)
            
        Returns:
            Reduced value
        """
        if initial is None:
            return reduce(func, self._iterator)
        else:
            return reduce(func, self._iterator, initial)
    
    def collect(self) -> List[T]:
        """
        Evaluate all operations and return results as a list.
        
        Returns:
            List containing all elements
        """
        return list(self._iterator)
    
    def first(self, predicate: Optional[Callable[[T], bool]] = None) -> Optional[T]:
        """
        Get the first element, optionally matching a predicate.
        
        Args:
            predicate: Optional condition to match
            
        Returns:
            First element or None if empty
        """
        for item in self._iterator:
            if predicate is None or predicate(item):
                return item
        return None
    
    def count(self) -> int:
        """
        Count the number of elements.
        
        Returns:
            Number of elements
        """
        return sum(1 for _ in self._iterator)
    
    def any_match(self, predicate: Callable[[T], bool]) -> bool:
        """
        Check if any element matches the predicate.
        
        Args:
            predicate: Condition to check
            
        Returns:
            True if any element matches
        """
        for item in self._iterator:
            if predicate(item):
                return True
        return False
    
    def all_match(self, predicate: Callable[[T], bool]) -> bool:
        """
        Check if all elements match the predicate.
        
        Args:
            predicate: Condition to check
            
        Returns:
            True if all elements match
        """
        for item in self._iterator:
            if not predicate(item):
                return False
        return True
    
    def paginate(self, page: int, page_size: int) -> 'LazyCollection[T]':
        """
        Paginate the results.
        
        Args:
            page: Page number (1-indexed)
            page_size: Number of items per page
            
        Returns:
            New LazyCollection for the requested page
        """
        skip_count = (page - 1) * page_size
        return self.skip(skip_count).take(page_size)
    
    def chunk(self, chunk_size: int) -> 'LazyCollection[List[T]]':
        """
        Split the collection into chunks of specified size.
        
        Args:
            chunk_size: Size of each chunk
            
        Returns:
            New LazyCollection of chunks
        """
        def chunk_iterator():
            chunk = []
            for item in self._iterator:
                chunk.append(item)
                if len(chunk) == chunk_size:
                    yield chunk
                    chunk = []
            if chunk:  # Yield remaining items
                yield chunk
        
        return LazyCollection(chunk_iterator())
    
    def distinct(self, key_func: Optional[Callable[[T], Any]] = None) -> 'LazyCollection[T]':
        """
        Remove duplicate elements.
        
        Args:
            key_func: Function to determine uniqueness (default: element itself)
            
        Returns:
            New LazyCollection with distinct elements
        """
        def distinct_iterator():
            seen = set()
            for item in self._iterator:
                key = key_func(item) if key_func else item
                if key not in seen:
                    seen.add(key)
                    yield item
        
        return LazyCollection(distinct_iterator())
    
    def zip_with(self, other: 'LazyCollection[U]') -> 'LazyCollection[tuple[T, U]]':
        """
        Zip with another LazyCollection.
        
        Args:
            other: Another LazyCollection to zip with
            
        Returns:
            New LazyCollection of tuples
        """
        def zip_iterator():
            iter1 = self._iterator
            iter2 = other._iterator
            while True:
                try:
                    item1 = next(iter1)
                    item2 = next(iter2)
                    yield (item1, item2)
                except StopIteration:
                    break
        
        return LazyCollection(zip_iterator())

In [4]:
def expensive_transformation(x: int) -> int:
    """Simulate an expensive computation."""
    print(f"Computing expensive transformation for {x}")
    return x * x

def expensive_filter(x: int) -> bool:
    """Simulate an expensive filter operation."""
    print(f"Checking filter condition for {x}")
    return x % 2 == 0

def main():
    print("=== Lazy Collection Demo ===\n")
    
    # Create a large dataset
    large_data = list(range(1, 1000001))  # 1 million items
    
    print("1. Basic lazy operations:")
    print("-" * 30)
    
    # Create lazy collection - no computation happens yet
    lazy_collection = LazyCollection(iter(large_data))
    
    # Chain multiple operations - still no computation
    result = (lazy_collection
             .map(expensive_transformation)  # Square each number
             .filter(expensive_filter)       # Keep even squares
             .take(5))                       # Take only first 5
    
    print("Operations chained, no computation yet...")
    print("Now collecting results (computation happens here):")
    
    # Computation happens only when we collect
    final_result = result.collect()
    print(f"Result: {final_result}")
    
    print("\n2. Pagination example:")
    print("-" * 30)
    
    # Simulate paginating through results
    page_size = 3
    total_pages = 3
    
    for page in range(1, total_pages + 1):
        print(f"\nPage {page}:")
        page_results = (LazyCollection(iter(range(1, 11)))  # 1-10
                       .map(lambda x: f"Item_{x}")
                       .paginate(page, page_size)
                       .collect())
        print(f"  {page_results}")
    
    print("\n3. Chunking example:")
    print("-" * 30)
    
    chunks = (LazyCollection(iter(range(1, 11)))
             .chunk(3)
             .collect())
    
    for i, chunk in enumerate(chunks, 1):
        print(f"Chunk {i}: {chunk}")
    
    print("\n4. Complex pipeline with reduction:")
    print("-" * 30)
    
    # Complex pipeline with multiple transformations
    total = (LazyCollection(iter(range(1, 101)))  # 1-100
            .filter(lambda x: x % 3 == 0)         # Multiples of 3
            .map(lambda x: x * 2)                 # Double them
            .distinct()                           # Remove duplicates (redundant here)
            .reduce(lambda acc, x: acc + x, 0))   # Sum all values
    
    print(f"Sum of doubled multiples of 3 from 1-100: {total}")
    
    print("\n5. Memory efficiency demonstration:")
    print("-" * 30)
    
    # This would use too much memory with regular lists
    # but works fine with lazy evaluation
    def infinite_numbers():
        num = 0
        while True:
            yield num
            num += 1
    
    # Process infinite stream lazily
    processed = (LazyCollection(infinite_numbers())
                .map(lambda x: x * 2)
                .filter(lambda x: x % 3 == 0)
                .take(10))
    
    print(f"First 10 even numbers divisible by 3 from infinite stream: {processed.collect()}")



In [5]:
if __name__ == "__main__":
    main()

=== Lazy Collection Demo ===

1. Basic lazy operations:
------------------------------
Operations chained, no computation yet...
Now collecting results (computation happens here):
Computing expensive transformation for 1
Checking filter condition for 1
Computing expensive transformation for 2
Checking filter condition for 4
Computing expensive transformation for 3
Checking filter condition for 9
Computing expensive transformation for 4
Checking filter condition for 16
Computing expensive transformation for 5
Checking filter condition for 25
Computing expensive transformation for 6
Checking filter condition for 36
Computing expensive transformation for 7
Checking filter condition for 49
Computing expensive transformation for 8
Checking filter condition for 64
Computing expensive transformation for 9
Checking filter condition for 81
Computing expensive transformation for 10
Checking filter condition for 100
Computing expensive transformation for 11
Checking filter condition for 121
Compu